# Library import

In [ ]:
import tensorflow as tf
import keras.api as keras
from keras.api import layers
from keras.api.layers import Conv2D, LeakyReLU, Flatten, Dropout, Dense
import matplotlib.pyplot as plt
import numpy as np

# Parameters

In [ ]:
image_size = (128, 128)
channel = 1
BUFFER_SIZE = 60000
BATCH_SIZE = 64
latent_dim_size = 1024
dataset_path = "../../data/train/"
complete_hist = {
    'loss_dis': [],
    'loss_gen': [],
}

# Functions

In [ ]:
def normalize(image):
    image = tf.cast(image, tf.float32) / 255.0
    return image

def loss_discriminador(real_output, fake_output):
    real_loss = keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def loss_gerador(fake_output):
    return keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

def create_generator():

    input = layers.Input((latent_dim_size,))
    x = layers.Dense(4*4*512, use_bias=True)(input)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((4, 4, 512))(x)

    x= layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU(.2)(x)
    

    x= layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU(.2)(x)
    
    x= layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU(.2)(x)

    x= layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU(.2)(x)
    
    x = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x= layers.BatchNormalization()(x)
    x = layers.LeakyReLU(.2)(x)

    x = Conv2D(1,3,1,'same', use_bias=True)(x)
    output = layers.Activation('sigmoid')(x)

    model = keras.Model(input,output)
    
    return model

def create_discriminator():

    input = layers.Input((128,128,1))
    
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same')(input)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1)(x)
    output = layers.Activation('sigmoid')(x)


    modelo = keras.Model(input,output)
    
    return modelo

# Dataset load

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    label_mode=None,
    color_mode='grayscale',
    image_size=image_size,
    shuffle=True,
    seed = 1567,
    batch_size=BATCH_SIZE
)
train_ds = train_ds.map(lambda x: (normalize(x)))

for batch in train_ds:
    for image in batch:
        plt.imshow(image,cmap=plt.cm.gray)
        plt.show()
        break
    break

# Create models

In [ ]:
gen = create_generator()
dis = create_discriminator()
gen.summary()

# Configurate optimizers

In [ ]:
gen_opt = keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5)
dis_opt = keras.optimizers.Adam(learning_rate=1e-4/2, beta_1=0.5)

# Training Function

In [ ]:
@tf.function
def train_step():
    gen_loss,dis_loss = 0.,0.
    gen_loss_iter,dis_loss_iter = 0.,0.
    for batch in train_ds:
        
        noise = tf.random.normal((BATCH_SIZE,latent_dim_size))

        with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
            fake_imgs = gen(noise,training=True)
            true_labels = dis(batch,training=True)
            fake_labels = dis(fake_imgs,training=True)

            gen_loss_iter = loss_gerador(fake_labels)
            dis_loss_iter = loss_discriminador(true_labels,fake_labels)
        
        gen_gras = gen_tape.gradient(gen_loss_iter,gen.trainable_variables)
        gen_opt.apply_gradients(zip(gen_gras,gen.trainable_variables))

        dis_grads = dis_tape.gradient(dis_loss_iter,dis.trainable_variables)
        dis_opt.apply_gradients(zip(dis_grads,dis.trainable_variables))

        gen_loss += gen_loss_iter
        dis_loss += dis_loss_iter
        gen_loss_iter,dis_loss_iter = 0.,0.

    return gen_loss/tf.cast(len(train_ds),tf.float32),dis_loss/tf.cast(len(train_ds),tf.float32)

# Training Block

In [ ]:
EPOCHS = 1000
EPOCH_SAMPLE = 10
n = 5

for i in range(EPOCHS):

    # Histórico de Loss
    loss_gen, loss_dis = train_step()
    complete_hist['loss_gen'].append(loss_gen)
    complete_hist['loss_dis'].append(loss_dis)
    
    # Iteração das épocas
    if i % EPOCH_SAMPLE == 0:
        # Print Loss
        print(f'Ep = {i} | Loss_gen = {loss_gen:.4f}; Loss_dis = {loss_dis:.4f}')
        # Salvar uma amostra das imagens
        noise = tf.random.normal((n**2,latent_dim_size))
        img_fake = gen(noise)
        fig, ax = plt.subplots(n,n,figsize=(1,1))
        ax = ax.ravel()
        for ii in range(n**2):
            ax[ii].imshow(img_fake[ii],cmap='gray')
            ax[ii].set_axis_off()
        plt.savefig(f'../../imgs_fake/fig{i}.png',dpi=1000)
        plt.close()

    plt.semilogy(np.array(complete_hist['loss_gen']),label=f'GEN = {loss_gen:.4f}',color='r')
    plt.semilogy(np.array(complete_hist['loss_dis']),label=f'DIS = {loss_dis:.4f}',color='k')
    plt.legend()
    plt.grid(True,'minor')
    plt.savefig('loss.png')
    plt.close()


print('==================== COMPLETE ====================')